In [1]:
from transformers import AutoTokenizer
from eval_dataset import FarsInstructEvalDataset

# checkpoint = "/media/abbas/Backup/mGPT-1.3B-persian/"
# checkpoint = "/media/abbas/Backup/aya"
# checkpoint = "/media/abbas/Backup/PersianMind-v1.0"
checkpoint = "/media/abbas/Backup/ava-llama3/"
path = "../data/1shot_instruct_dataset_test_entailment-sentiment-pharaphrase.csv"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side="left")

ds = FarsInstructEvalDataset(tokenizer, 200, 'none', 'test', shots=1, path=path)
encoded_ds = ds.get_tokenized_data('PNLPhub/FarsTail', 'confidence', multiple_choice=False)


In [2]:
encoded_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})

In [3]:
print(tokenizer.decode(encoded_ds['input_ids'][0]))
print(tokenizer.decode(encoded_ds['attention_mask'][0]))
print(tokenizer.decode(encoded_ds['labels'][0]))

<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>سطح اطمینان خود را در شباهت عبارات ارائه شده بیان کنید

عبارت اول: کارستن نیبور شرق شناس و باستان شناس آلمانی اولین کسی بود که در سال ۱۱۴۴ هجری خورشیدی از تخت جمشید و نواحی جنوبی و مرکزی ایران بازدید کرد، وی با بررسی تخت جمشید، برای اولین بار گزارشهای بسیار مهم علمی از تخت جمشید ارائه داد.
عبارت دوم: گروتفند نخستین مورخی است که از تخت جمشید بازدید کرد و اولین گزارش علمی را از تخت جمشید ارائه داد.


In [4]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer,
                                        return_tensors='pt')
        
val_dataloader = DataLoader(encoded_ds, collate_fn=data_collator, batch_size=8)

In [5]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from accelerate import Accelerator
import torch


device = "cuda" if torch.cuda.is_available() else "cpu"

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

try:
    model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                                quantization_config=quantization_config,
                                                torch_dtype=torch.float16,
                                                device_map="auto"
    )
except:
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,
                                                quantization_config=quantization_config,
                                                torch_dtype=torch.float16,
                                                device_map="auto"
    )

accelerator = Accelerator(cpu=False)

val_dataloader, model = accelerator.prepare(val_dataloader, model)

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
model.name_or_path

'/media/abbas/Backup/ava-llama3/'

In [7]:
import torch 
import numpy as np
import datasets
from hazm import sent_tokenize


def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(sent_tokenize(label)) for label in labels]

        return preds, labels


def evall():
    dec_preds = []
    dec_labels = []
    model.eval()
    metric = datasets.load_metric('rouge')
    for step, batch in enumerate(val_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_new_tokens=512
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]
            
            generated_tokens, labels = accelerator.gather_for_metrics((generated_tokens, labels))
            generated_tokens = generated_tokens.cpu().numpy()
            labels = labels.cpu().numpy()

            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            # decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            # if model.name_or_path != "/media/abbas/Backup/aya":
            #     decoded_preds = tokenizer.batch_decode(generated_tokens[:, batch['input_ids'].shape[1]:])
            # else:
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )

            # print(decoded_preds)
            # print(tokenizer.batch_decode(batch['input_ids']))

            dec_preds = decoded_preds
            dec_labels = decoded_labels

    scores = metric.compute(
        rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"], 
        use_stemmer=False, 
        lang='fa')

    from prettytable import PrettyTable
    table = PrettyTable(title='Rouge Metric')
    table.field_names = ["Score", "Type", "Precision", "Recall", "F-Measure"]
    for i, score_name in enumerate(scores):
        score = dict(scores[score_name]._asdict())
        for score_type in score:
            values = score[score_type]
            table.add_row([f'{score_name}', f'{score_type}', f'{values.precision * 100:0.2f} ', f'{values.recall * 100:0.2f}', f'{values.fmeasure * 100:0.2f}'])

        if i < len(scores) - 1:
            table.add_row([''] * len(table.field_names))
            table.add_row(['***'] * len(table.field_names))
            table.add_row([''] * len(table.field_names))

    rouge_table_valid = table
    print(table.get_string())

    return dec_preds, dec_labels
    # result = {k: round(v * 100, 4) for k, v in result.items()}


## Farstail -- can you infer

In [9]:
# aya
evall()

/tmp/ipykernel_2425540/1637929580.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   38.00   | 38.00  |   38.00   |
|   rouge1  | mid  |   45.50   | 45.50  |   45.50   |
|   rouge1  | high |   52.50   | 52.50  |   52.50   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   39.00   | 39.00  |   39.00   |
|   rougeL  | mid  |   45.50

(['بله', 'بله', 'بله', 'بله', 'بله', 'خیر.', 'خیر.', 'خیر.'],
 ['خیر',
  'نمیتوان نظر قطعی داد',
  'بله',
  'نمیتوان نظر قطعی داد',
  'بله',
  'نمیتوان نظر قطعی داد',
  'نمیتوان نظر قطعی داد',
  'خیر'])

In [46]:
# persian_mind
evall()

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   12.79   | 28.50  |   17.37   |
|   rouge1  | mid  |   15.71   | 34.00  |   21.24   |
|   rouge1  | high |   19.00   | 41.00  |   25.62   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   12.58   | 27.50  |   16.97   |
|   rougeL  | mid  |   15.63

(['خیر</s>',
  'خیر</s>',
  'ب</s>',
  'خیر</s>',
  'خیر</s>',
  'خیر</s>',
  'خیر</s>',
  'خیر</s>'],
 ['خیر',
  'نمیتوان نظر قطعی داد',
  'بله',
  'نمیتوان نظر قطعی داد',
  'بله',
  'نمیتوان نظر قطعی داد',
  'نمیتوان نظر قطعی داد',
  'خیر'])

In [54]:
# mgpt 
evall()

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Set

+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   0.19    | 23.75  |    0.38   |
|   rouge1  | mid  |   0.24    | 30.12  |    0.48   |
|   rouge1  | high |   0.30    | 36.50  |    0.58   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   0.19    | 23.75  |    0.38   |
|   rougeL  | mid  |   0.25 

(['بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\

In [ ]:
# tk-instruct

In [61]:
# ava
evall()

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   1.66    | 44.38  |    2.75   |
|   rouge1  | mid  |   2.62    | 50.88  |    4.25   |
|   rouge1  | high |   3.81    | 57.50  |    6.08   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   1.67    | 43.50  |    2.77   |
|   rougeL  | mid  |   2.58 

(['بله<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>',
  'بله.\nبر اساس عبارت اول، نثر مرسل نوعی نثر است که ساده و از پیرایه\u200cها و آ

In [12]:
# ava with <|temp|>
evall()

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   2.24    | 100.00 |    4.31   |
|   rouge1  | mid  |   2.50    | 100.00 |    4.82   |
|   rouge1  | high |   2.78    | 100.00 |    5.34   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   1.07    | 28.50  |    2.05   |
|   rouge2  | mid  |   1.30    | 35.00  |    2.51   |
|   rouge2  | high |   1.57    | 41.51  |    3.02   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   2.24    | 100.00 |    4.33   |
|   rougeL  | mid  |   2.51 

(['آیا براساس عبارت اول میتوان عبارت دوم را استنتاج کرد؟\nجواب را از بین گزینه های داده شده انتخاب کن - بله - خیر - نمیتوان نظر قطعی داد\nعبارت اول: بُرهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nجواب: <im_start>',
  'آیا براساس عبارت اول میتوان عبارت دوم را استنتاج کرد؟\nجواب را از بین گزینه های داده شده انتخاب کن - بله - خیر - نمیتوان نظر قطعی داد\nعبارت اول: نثر مرسل که با نام نثر ساده نیز عنوان می شود، نوشته\u200cای است ساده و از پیرایه\u200cها و آرایش\u200cهای ادبی خالی است.\nعبارت دوم: نوع نثر تاریخ بیهقی، نثر مرسل\u200c است.\nجواب: <im_start>',
  'آیا براساس عبارت اول میتوان عبارت دوم را استنتاج کرد؟\nجواب را از بین گزینه های داده شده انتخاب کن - بله - خیر - نمیتوان نظر قطعی داد\nعبارت اول: رودابه نام دختر مهراب کابلی و سیندخت است او همسر زال و مادر رستم(شخصیت اسطوره ای شاهنامه) می\u200cباشد.\nزال در گذری از مرز کابل، به رودابه علاقه\u200cمند شد و او را به 

In [14]:
# ava with temp
evall()

/tmp/ipykernel_2429477/3065205785.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/hom

+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   1.43    | 100.00 |    2.81   |
|   rouge1  | mid  |   1.47    | 100.00 |    2.90   |
|   rouge1  | high |   1.52    | 100.00 |    3.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   1.43    | 100.00 |    2.81   |
|   rougeL  | mid  |   1.47 

(['Human:آیا جمله دوم به جمله اول مرتبط است؟\nصحیح، غلط، شاید\nپیش فرض: بُرهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nفرضیه: برهان فطرت نام دیگر برهان اتقان صنع است.\nجواب: <im_start>Assistant:',
  'Human:آیا جمله دوم به جمله اول مرتبط است؟\nصحیح، غلط، شاید\nپیش فرض: نثر مرسل که با نام نثر ساده نیز عنوان می شود، نوشته\u200cای است ساده و از پیرایه\u200cها و آرایش\u200cهای ادبی خالی است.\nفرضیه: نوع نثر تاریخ بیهقی، نثر مرسل\u200c است.\nجواب: <im_start>Assistant:',
  'Human:آیا جمله دوم به جمله اول مرتبط است؟\nصحیح، غلط، شاید\nپیش فرض: رودابه نام دختر مهراب کابلی و سیندخت است او همسر زال و مادر رستم(شخصیت اسطوره ای شاهنامه) می\u200cباشد.\nزال در گذری از مرز کابل، به رودابه علاقه\u200cمند شد و او را به همسری برگزید.\nفرضیه: نام مادر رستم در شاهنامه، رودابه بود.\nجواب: <im_start>Assistant:آیا جمله دوم به جمله اول مرتبط است؟\nصحیح، غلط، شاید\nپیش فرض: رودابه نام دختر مهراب کابلی و سیندخت است او همسر زال و مادر رستم(شخصی

In [8]:
evall()

/tmp/ipykernel_2433421/3065205785.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/hom

+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   0.22    | 29.00  |    0.44   |
|   rouge1  | mid  |   0.26    | 32.50  |    0.52   |
|   rouge1  | high |   0.31    | 35.50  |    0.61   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   0.23    | 29.24  |    0.45   |
|   rougeL  | mid  |   0.27 

(['سطح اطمینان خود را در شباهت عبارات ارائه شده بیان کنید\nعبارت اول: بُرهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nسطح اطمینان: 0.8\nعبارت اول: برهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nسطح اطمینان: 0.8\nعبارت اول: برهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nسطح اطمینان: 0.8\nعبارت اول: برهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nسطح اطمینان: 0.8\nعبارت اول: برهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان